In [2]:
# Unified Intent Detection + Entity Extraction using FLAN-T5

!pip install -q transformers datasets accelerate

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq
from datasets import Dataset
import pandas as pd
import torch

In [3]:
# 1. Sample Data
examples = [
    {"input": "Send 500 to Arun", "target": '{"intent": "money_transfer", "recipient": "Arun", "amount": 500}'},
    {"input": "Book a cab to airport", "target": '{"intent": "book_ride", "destination": "airport"}'},
    {"input": "Recharge my phone with 199", "target": '{"intent": "recharge", "amount": 199}'},
    {"input": "Pay electricity bill", "target": '{"intent": "bill_payment", "type": "electricity"}'},
    {"input": "Transfer 1000 to Neha", "target": '{"intent": "money_transfer", "recipient": "Neha", "amount": 1000}'}
]

# Convert to HF dataset
df = pd.DataFrame(examples)
dataset = Dataset.from_pandas(df)

In [5]:
# 2. Tokenizer and Model
model_checkpoint = "google/flan-t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [6]:
# 3. Tokenization Function
def preprocess(example):
    model_inputs = tokenizer(example['input'], max_length=64, truncation=True)
    labels = tokenizer(example['target'], max_length=64, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_ds = dataset.map(preprocess)

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

In [9]:
# 4. Training Arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./flan_intent_model",
    per_device_train_batch_size=4,
    num_train_epochs=10,
    logging_dir="./logs",
    logging_steps=1,
    eval_strategy="no",
    save_strategy="no",
    predict_with_generate=True,
    fp16=torch.cuda.is_available(),
)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [10]:
# 5. Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

<ipython-input-10-3101599487>:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


# New Section

# New Section

In [11]:
# 6. Train
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: it-nitinn (it-nitinn-mbm-news-network) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss
1,3.903700
2,4.177200
3,3.593700
4,3.104200
5,3.613300
6,2.959600
7,3.513000
8,3.062700
9,3.188400
10,3.854600


TrainOutput(global_step=20, training_loss=3.2756590723991392, metrics={'train_runtime': 233.2552, 'train_samples_per_second': 0.214, 'train_steps_per_second': 0.086, 'total_flos': 140870148096.0, 'train_loss': 3.2756590723991392, 'epoch': 10.0})

In [14]:
# 7. Inference
def predict(text):
    inputs = tokenizer(text, return_tensors="pt").to(model.device)
    outputs = model.generate(**inputs, max_length=64)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# 🔍 Try it
print(predict("Venmo 200 to Anita"))
print(predict("Recharge Jio with 299"))

                               
Recharge Jio with 256 knots
